# Complexidade de letras:

## Introdução:

Este código foi desenvolvido com o intuito de explorar e revelar a complexidade das letras musicais, proporcionando uma análise detalhada de sua riqueza lírica. Diversos critérios foram considerados para avaliar a profundidade e sofisticação das composições, incluindo a diversidade temática das letras. A seguir, estão os principais critérios utilizados na análise:

Número de sílabas: Palavras com mais sílabas geralmente indicam maior complexidade, tanto em termos de compreensão quanto de pronúncia.

Frequência de palavras: A frequência com que as palavras aparecem na letra pode sugerir sua acessibilidade e familiaridade para o público.

Análise gramatical: Frases com muitos gerúndios tendem a ser mais coloquiais, enquanto aquelas com maior uso de conjunções subordinadas indicam um tom mais formal.

Diversidade lexical: A variedade de vocabulário e a riqueza semântica contribuem significativamente para a expressividade e profundidade das letras.

Diversidade temática: A amplitude dos temas abordados nas letras reflete sua complexidade, com composições que exploram uma gama mais ampla de assuntos sendo consideradas mais ricas e elaboradas.

Tamanho da letra: A extensão da letra, medida pelo número de palavras, pode indicar o nível de desenvolvimento da composição.

Tamanho dos versos: Versos mais longos ou mais curtos influenciam a fluidez e o ritmo das músicas, impactando também sua estrutura.

Polaridade: A análise de polaridade refere-se à avaliação do tom emocional da letra, classificando-a como positiva, negativa ou neutra.

Para concluir, a classificação da complexidade das letras será realizada por meio do modelo `Random Forest`, que integra esses critérios, permitindo uma avaliação abrangente da profundidade e sofisticação das composições musicais. Essa abordagem visa não apenas quantificar a complexidade lírica, mas também oferecer informações sobre a riqueza e diversidade temática presente nas letras

##  2 - Desenvolvimento:

### 2.1 - Importando bibliotecas:

In [968]:
import pandas as pd
import textstat as tst
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import numpy as np
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from textblob import TextBlob

### 2.2 - Leitura do dataset:

Nessa seção carregamos nosso dataset. Como nosso intuito é apresentar como esse código funciona, limitaremos nosso dataset a 50 linhas.

In [972]:
df_carregado = pd.read_csv("datasets/dataset_atualizado.csv")
df = df_carregado.head(50)

### 2.3 - Descrição do dataset utilizado:

Artist: Artista

Track_name: Nome da música

Lyrics: Letra da música

genre: Gênero musical

In [973]:
df.head(5)

,artist,track_name,lyrics,genre
0,abba,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \nAnd...",pop
1,abba,"Andante, Andante","Take it easy with me, please \nTouch me gentl...",pop
2,abba,As Good As New,I'll never know why I had to go \nWhy I had t...,pop
3,abba,Bang,Making somebody happy is a question of give an...,pop
4,abba,Bang-A-Boomerang,Making somebody happy is a question of give an...,pop


### 2.4 - Limpeza de dados:

Como são letras de música, as linhas contêm \r ou \n, o que, futuramente, pode ocasionar problemas de entendimento, sendo necessário filtrá-las.

In [974]:
def limpar_texto(texto):
    texto = texto.replace("\r\n", " ").replace("\n", " ").replace("<br/>", "\n").replace("<p>", " ").replace("</p>", "\n").strip().lower()
    return texto

Além disso, criamos uma nova coluna com as letras "limpas". Não substituímos na coluna "lyrics", pois, futuramente, iremos necessitar dos "\n" para calcular o tamanho dos versos

In [ ]:
df['letras_limpas'] = df['lyrics'].apply(limpar_texto)

Indexar os nomes das músicas para evitar que músicas de nomes iguais, mas de contextos diferentes sejam confundidas:

In [ ]:
df['track_name'] = [f"{index} - {name}" for index, name in enumerate(df['track_name'])]

In [978]:
df.head(5)

,artist,track_name,lyrics,genre,letras_limpas
0,abba,0 - Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \nAnd...",pop,"look at her face, it's a wonderful face and ..."
1,abba,"1 - Andante, Andante","Take it easy with me, please \nTouch me gentl...",pop,"take it easy with me, please touch me gently..."
2,abba,2 - As Good As New,I'll never know why I had to go \nWhy I had t...,pop,i'll never know why i had to go why i had to...
3,abba,3 - Bang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...
4,abba,4 - Bang-A-Boomerang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...


## 3 - Análises exploratórias:

Nessa parte do projeto, focaremos em procurar ferramentas que facilitem nossa classificação de complexidade das letras musicais

### 3.1 - Número de sílabas:

Este parte do projeto tem como objetivo contar o número de sílabas de cada palavra em letras de músicas e identificar palavras que possuem uma certa complexidade lírica. Para isso, utilizaremos uma biblioteca chamada `textstat`, que tem uma função chamada `syllable_count`, a qual calcula a quantidade de sílabas nas palavras

### Passo 1: Criar um Dicionário de Sílaba

Primeiro, criamos um dicionário que armazena o número de sílabas de cada palavra em cada música. Usaremos dictionary comprehension, em que o nome da música será a key e a quantidade de sílabas de cada palavra será o Value. 

In [979]:
dic_silabas = {
    track: {palavra: tst.syllable_count(palavra) for palavra in letras.split(" ")} 
    for track, letras in zip(df['track_name'], df['letras_limpas']) #Nesse caso, o zip é utilizado para associar o "track_name" e "letras_limpas"
}

### Passo 2: Contar Palavras Complexas

Aqui, percorremos as músicas e suas palavras para identificar palavras que possuem 5 ou mais sílabas. Também filtramos expressões repetitivas para uma análise mais precisa.

In [980]:
# Dicionário para armazenar a contagem de palavras com 4 ou mais sílabas em cada música
musicas_silabas_complexas = {}

for nome_musica, palavras in dic_silabas.items():
    for palavra, silabas in palavras.items():
        # Verifica se a palavra tem 4 ou mais sílabas
        if silabas >= 5:
            # Filtra palavras que contêm uma letra quatro ou mais vezes (ex: "Ahhhhhhh"), comum em músicas
            if not any(palavra.count(letra) >= 4 for letra in set(palavra)):
                if nome_musica not in musicas_silabas_complexas:
                    musicas_silabas_complexas[nome_musica] = 1
                else:
                    musicas_silabas_complexas[nome_musica] += 1

### Passo 3: Criar uma coluna no dataset

Aqui, criaremos uma coluna referente a quantidade de sílabas maiores que 5 que a letra possui e preenchemos as outras com 0.

In [ ]:
df['Silabas'] = df['track_name'].map(musicas_silabas_complexas).fillna(0)

In [982]:
df.head(5)

,artist,track_name,lyrics,genre,letras_limpas,Silabas
0,abba,0 - Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \nAnd...",pop,"look at her face, it's a wonderful face and ...",0.0
1,abba,"1 - Andante, Andante","Take it easy with me, please \nTouch me gentl...",pop,"take it easy with me, please touch me gently...",0.0
2,abba,2 - As Good As New,I'll never know why I had to go \nWhy I had t...,pop,i'll never know why i had to go why i had to...,0.0
3,abba,3 - Bang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...,0.0
4,abba,4 - Bang-A-Boomerang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...,0.0


### 3.2 - Frequência:

Esta parte do projeto visa calcular a diversidade lexical das letras das músicas, utilizando a medida TTR (Type-Token Ratio), que é a razão entre o número de palavras únicas e o total de palavras em uma letra. 

### Passo 1: Calcular o TTR (Type-Token Ratio)

Aqui, calculamos o TTR para cada letra, dividindo o número de palavras únicas pelo total de palavras na letra. O resultado é armazenado em uma lista para análise posterior.

In [983]:
lista_ttr = [ len(set(df.letras_limpas[letra].lower().split())) / len(df.letras_limpas[letra].split())
    if len(df.letras_limpas[letra].split()) > 0 else 0
    for letra in range(len(df))
]

### Passo 2: Criar uma coluna no dataset

In [ ]:
df["Frequencia"] = lista_ttr

In [986]:
df.head(5)

,artist,track_name,lyrics,genre,letras_limpas,Silabas,Frequencia
0,abba,0 - Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \nAnd...",pop,"look at her face, it's a wonderful face and ...",0.0,0.457516
1,abba,"1 - Andante, Andante","Take it easy with me, please \nTouch me gentl...",pop,"take it easy with me, please touch me gently...",0.0,0.307692
2,abba,2 - As Good As New,I'll never know why I had to go \nWhy I had t...,pop,i'll never know why i had to go why i had to...,0.0,0.362179
3,abba,3 - Bang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...,0.0,0.440000
4,abba,4 - Bang-A-Boomerang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...,0.0,0.454545


### 3.3 - Análise Gramatical:

Este código tem como objetivo calcular uma pontuação com base nas tags gramaticais das palavras presentes nas letras das músicas. A pontuação é atribuída de acordo com a presença de substantivos, adjetivos, advérbios, verbos e outras categorias gramaticais. Para realizar a análise gramatical, utilizamos a biblioteca NLTK (Natural Language Toolkit), que fornece ferramentas para a tokenização e a análise de partes do discurso (POS tagging).

### Passo 1: Inicializar o Dicionário de Pontuação

Primeiramente, criamos um dicionário (`dic_tags`) onde cada música é uma chave e sua pontuação inicial é zero. Isso nos permitirá acumular a pontuação ao longo da análise.

In [988]:
dic_tags = {track: 0 for track in df['track_name']} 

### Passo 2: Iterar Sobre as Letras e Calcular a Pontuação

Neste passo, utilizamos um loop para percorrer cada letra do DataFrame. Para cada letra, realizamos a tokenização e a análise de partes do discurso (POS tagging) para identificar a categoria gramatical de cada palavra. Em seguida, atribuímos uma pontuação com base nas tags gramaticais.

In [989]:
for index, row in df.iterrows():
    tokens = word_tokenize(row['letras_limpas'].lower())  # Tokeniza a letra em palavras
    tags = nltk.pos_tag(tokens) 
    
    pontuacao = 0  
    for palavra, tag in tags:
        # Atribui pontuação com base nas tags gramaticais
        if tag in ['NNP', 'NNPS']:  # Substantivos próprios
            pontuacao += 0.02
        elif tag in ['JJ', 'JJR', 'JJS']:  # Adjetivos
            pontuacao += 0.01
        elif tag in ['RB', 'RBR', 'RBS']:  # Advérbios
            pontuacao += 0.01
        elif tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:  # Verbos
            pontuacao += 0.01
        elif tag == 'WDT':  # Determinante interrogativo
            pontuacao += 0.01 
        elif tag == 'IN':  # Preposição ou conjunção subordinativa 
            pontuacao += 0.01
            
    dic_tags[row['track_name']] = pontuacao 

### Passo 3: Criar uma coluna no dataset

In [ ]:
df['Pontuacao Gramatical'] = df['track_name'].map(dic_tags)

In [991]:
df.head(5)

,artist,track_name,lyrics,genre,letras_limpas,Silabas,Frequencia,Pontuacao Gramatical
0,abba,0 - Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \nAnd...",pop,"look at her face, it's a wonderful face and ...",0.0,0.457516,0.71
1,abba,"1 - Andante, Andante","Take it easy with me, please \nTouch me gentl...",pop,"take it easy with me, please touch me gently...",0.0,0.307692,1.32
2,abba,2 - As Good As New,I'll never know why I had to go \nWhy I had t...,pop,i'll never know why i had to go why i had to...,0.0,0.362179,1.74
3,abba,3 - Bang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...,0.0,0.440000,1.03
4,abba,4 - Bang-A-Boomerang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...,0.0,0.454545,1.06


### 3.4 - Diversidade Lexical:

Esta parte do projeto tem como objetivo calcular a complexidade das letras das músicas usando o modelo Word2Vec. A complexidade é determinada pela variância dos vetores das palavras, permitindo uma análise da riqueza e diversidade vocabular presente nas letras. A variância é utilizada porque indica o grau de dispersão dos vetores em relação à média, refletindo a diversidade vocabular: uma maior variância sugere um vocabulário mais rico e variado, enquanto uma menor variância indica similaridade entre as palavras, resultando em uma letra potencialmente menos complexa.

### Passo 1: Remoção de Palavras Comuns (Stop Words)

Primeiro, criamos um conjunto de palavras comuns (stop words) que serão removidas das letras, uma vez que essas palavras geralmente não contribuem para a variabilidade temática.

In [992]:
stop_words = set(stopwords.words('english')) 

### Passo 2: Treinamento do Modelo Word2Vec

Em seguida, utilizamos o Word2Vec para criar um modelo que representa as palavras em um espaço vetorial. Isso nos permitirá calcular a complexidade das letras com base nos vetores gerados. Como, postariormente, iremos treinar um modelo para classificar se a música é complexa ou não, e esse dataset de músicas complexas não contêm muitas músicas, precisamos dividir o modelo em fatias para que a análise seja feita em proporções iguais. A função `tokenize_text` tem como função prevenir que músicas grandes sejam prejudicadas, já que o `Word2Vec` utiliza coocorrência para atribuir valores às palavras.

In [996]:
num_fatias = 1 #Altere o número de fatias de acordo com a necessidade

tamanho_fatia = len(df) // num_fatias  

# Função para dividir o texto em sentenças e depois em palavras
def tokenize_text(text):
    sentences = nltk.sent_tokenize(text)  # Divide o texto em sentenças
    return [sentence.split() for sentence in sentences]  # Divide cada sentença em palavras

# Treinar um modelo Word2Vec para cada fatia
modelos = []

for i in range(num_fatias):
    fatia = df.iloc[i * tamanho_fatia:(i + 1) * tamanho_fatia]
    
    # Aplicar a tokenização nas letras
    tokenized_letras = fatia['letras_limpas'].apply(tokenize_text)
    
    tokenized_letras_flat = [word for sublist in tokenized_letras for sentence in sublist for word in sentence]

    # Treinar o modelo Word2Vec na fatia
    model = Word2Vec([tokenized_letras_flat], vector_size=50, window=3, min_count=1, workers=2)
    
    modelos.append(model)

### Passo 3: Definição das Funções

#### Passo 4.1 - Cálculo da Variância dos Vetores

A função `calcular_variancia_vetores` tem como objetivo calcular a soma das variâncias dos vetores correspondentes a uma lista de palavras, utilizando Word2Vec.

- Passos:
  1. Para cada palavra na lista, o código verifica se ela está presente no vocabulário do modelo.
  2. Se a palavra estiver no modelo, o vetor correspondente à palavra é adicionado a uma lista.
  3. Após coletar os vetores, o código os normaliza utilizando `MinMaxScaler`.
  4. Em seguida, calcula a variância ao longo do eixo 0 (variância de cada dimensão dos vetores).
  5. Retorna a soma dessas variâncias.

In [997]:
def calcular_variancia_vetores(palavras, modelo):
    vetores = []
    for palavra in palavras:
        if palavra in modelo.wv:  # Verifica se a palavra está no vocabulário do modelo
            vetores.append(modelo.wv[palavra])  
    
    if len(vetores) > 0:  # Evitar calcular variância em lista vazia
        vetores_np = np.array(vetores)  # Converte a lista de vetores em um array NumPy

        scaler = MinMaxScaler()
        vetores_normalizados = scaler.fit_transform(vetores_np)
        
        variancia = np.var(vetores_normalizados, axis=0)  # Calcula a variância ao longo do eixo 0
        
        return np.sum(variancia)  
        
    else:
        return 0 


#### Passo 4.2 - Cálculo da Complexidade da Letra

A função `calcular_complexidade` processa a letra da música e calcula a sua complexidade utilizando a função `complexidade_vetores`.

In [998]:
def avaliar_complexidade_letra(letra, modelo):
    # Remove stopwords e processa a letra
    letra_processada = [palavra.lower() for palavra in letra.split() if palavra not in stop_words]
    return calcular_variancia_vetores(letra_processada, modelo)

### Passo 5 - Cálculo da Complexidade para Cada Música

Aqui, inicializamos um dicionário `dic_complexidade` para armazenar a complexidade de cada música e iteramos sobre as letras para calcular essa complexidade.

In [999]:
dic_complexidade = {df.track_name[n]: {} for n in range(len(df))} 

for i in range(num_fatias):
    fatia = df.iloc[i * tamanho_fatia:(i + 1) * tamanho_fatia]
    modelo_index = i # Para fazer a letra ser carregada no seu modelo correspondente
    for index, row in fatia.iterrows():
        letra = row['letras_limpas']
        complexidade = avaliar_complexidade_letra(letra, modelos[modelo_index])  
        dic_complexidade[df.track_name[index]] = complexidade  

lista_div = [dic_complexidade[df.track_name[n]] for n in range(len(df))]

### Passo 6 - Criar uma coluna no dataset:

In [ ]:
df['diversidade lexical'] = lista_div

In [1002]:
df.head(5)

,artist,track_name,lyrics,genre,letras_limpas,Silabas,Frequencia,Pontuacao Gramatical,diversidade lexical
0,abba,0 - Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \nAnd...",pop,"look at her face, it's a wonderful face and ...",0.0,0.457516,0.71,2.848807
1,abba,"1 - Andante, Andante","Take it easy with me, please \nTouch me gentl...",pop,"take it easy with me, please touch me gently...",0.0,0.307692,1.32,3.070545
2,abba,2 - As Good As New,I'll never know why I had to go \nWhy I had t...,pop,i'll never know why i had to go why i had to...,0.0,0.362179,1.74,2.994859
3,abba,3 - Bang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...,0.0,0.440000,1.03,3.135562
4,abba,4 - Bang-A-Boomerang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...,0.0,0.454545,1.06,3.193389


### 3.5 - Diversidade temática

Este código tem como objetivo analisar a presença de temas nas letras das músicas, utilizando um modelo Word2Vec para calcular a similaridade entre os vetores das palavras da letra e os vetores de temas predefinidos. A seguir, descrevemos as principais etapas do processo:

### Passo 1: Definição de Temas e Palavras-chave

In [1003]:
themes = {
    'love': ['love', 'affection', 'heart', 'passion', 'desire', 'romance', 'relationship', 'devotion'],
    'sadness': ['sadness', 'pain', 'longing', 'loneliness', 'lament', 'loss', 'grief', 'heartbreak'],
    'freedom': ['freedom', 'live', 'free', 'independence', 'choice', 'escape', 'liberation', 'release'],
    'friendship': ['friendship', 'companion', 'loyalty', 'trust', 'camaraderie', 'brotherhood', 'sisterhood'],
    'hope': ['hope', 'faith', 'dream', 'future', 'light', 'renewal', 'aspiration', 'optimism'],
    'nature': ['nature', 'earth', 'sky', 'sea', 'forest', 'wildlife', 'environment', 'sustainability'],
    'struggle': ['struggle', 'battle', 'strength', 'resistance', 'challenge', 'conquest', 'perseverance', 'determination'],
    'joy': ['joy', 'happiness', 'laughter', 'smile', 'celebration', 'fun', 'excitement', 'pleasure'],
    'solitude': ['solitude', 'isolation', 'distance', 'disconnection', 'emptiness', 'loneliness', 'reflection'],
    'nostalgia': ['nostalgia', 'memories', 'past', 'remembrance', 'reminiscence', 'sentimentality', 'yearning'],
    'change': ['change', 'transformation', 'growth', 'evolution', 'new beginnings', 'adaptation', 'progress'],
    'self-love': ['self-love', 'self-esteem', 'self-awareness', 'acceptance', 'confidence', 'self-care', 'nurturing'],
    'conflict': ['conflict', 'dispute', 'disagreement', 'tension', 'polarization', 'war', 'strife', 'debate'],
    'solidarity': ['solidarity', 'help', 'support', 'community', 'union', 'togetherness', 'cooperation'],
    'socio-political': ['socio-political', 'justice', 'rights', 'freedom of speech', 'equality', 'activism', 'revolution'],
    'party': ['party', 'celebration', 'fun', 'dance', 'joy', 'gathering', 'festivity', 'nightlife'],
    'mental health': ['mental health', 'anxiety', 'depression', 'healing', 'balance', 'well-being', 'mindfulness'],
    'identity': ['identity', 'self', 'cultural', 'heritage', 'roots', 'belonging', 'diversity'],
    'wealth': ['wealth', 'money', 'success', 'prosperity', 'luxury', 'status', 'ambition'],
    'addiction': ['addiction', 'dependency', 'substance', 'struggle', 'recovery', 'temptation'],
    'family': ['family', 'blood', 'kinship', 'upbringing', 'legacy', 'generation'],
    'betrayal': ['betrayal', 'deceit', 'trust', 'disloyalty', 'treachery', 'backstab'],
}


### Passo 2: Cálculo da Média dos Vetores de Palavras

A função media_vetores recebe uma lista de palavras e o modelo Word2Vec, retornando a média dos vetores dessas palavras. Isso permite representar um conjunto de palavras como um único vetor, que captura a essência do significado coletivo.

In [1004]:
def media_vetores(palavras, modelo):
    vetores = []
    for palavra in palavras:
        if palavra in modelo.wv:
            vetores.append(modelo.wv[palavra])
    
    if len(vetores) == 0:
        return np.zeros(modelo.vector_size)  # Retorna um vetor de zeros se não houver palavras válidas
    
    return np.mean(vetores, axis=0)

### Passo 3: Normalizar

Para evitar grandes diferenças entre datasets de tamanhos diferentes, já que a normalização remove o impacto da magnitude (tamanho) dos vetores e permite que a similaridade seja determinada com base na direção dos vetores, não apenas no seu comprimento

In [1005]:
def normalizar_similaridade(similaridade, vetor_letra):
    # Normaliza pela magnitude do vetor letra
    magnitude_letra = np.linalg.norm(vetor_letra)
    
    if magnitude_letra != 0:
        return similaridade / magnitude_letra
        
    return similaridade

### Passo 4: Cálculo da Similaridade Temática

A função resultado é responsável por comparar a letra da música (representada por seu vetor) com os vetores de cada tema. Para cada tema, a função:

a) Calcula o vetor médio das palavras-chave do tema.

Motivação: O vetor médio representa a essência do tema, capturando a semântica das palavras associadas a ele. Isso permite que a análise considere o contexto das palavras, facilitando a comparação com a letra.

b) Compara a similaridade (usando a similaridade cosseno) entre o vetor da letra e o vetor do tema.

Motivação: A similaridade cosseno mede a proximidade entre dois vetores, indicando quão semelhantes são os significados representados. Essa comparação ajuda a identificar a relação temática entre a letra da música e os temas predefinidos.

In [1006]:
def resultado(themes, model, vetor_letra):
    resultado = {}
    for tema, palavras_chave in themes.items():
        
        #Calcula o vetor médio para o tema baseado nas palavras-chave associadas
        vetor_tema = media_vetores(palavras_chave, model)

        #Verifica se tanto o vetor do tema quanto o vetor da letra existem e se contêm valores válidos
        if vetor_tema is not None and vetor_letra is not None and np.any(vetor_tema) and np.any(vetor_letra): #np.any assegura que os vetores não sejam vetores nulos ou cheios de zeros
            similaridade = model.wv.cosine_similarities(vetor_letra, np.array([vetor_tema]))
            
            # Aplica a normalização da similaridade
            similaridade_normalizada = normalizar_similaridade(similaridade[0], vetor_letra)
            resultado[tema] = similaridade_normalizada
            
        else:
            resultado[tema] = 0
    
    return resultado

### Passo 5: Iteração Sobre as Letras das Músicas

O código itera sobre cada linha do DataFrame df, processando as letras das músicas:

a) Converte as letras para minúsculas e remove palavras de parada (stop words).

b) Calcula o vetor da letra usando a função media_vetores.

c) Chama a função resultado para obter a similaridade entre a letra e os temas, armazenando os resultados em um dicionário chamado dic_div_tematica.

In [1007]:
# Cria um dicionário vazio para armazenar a diversidade temática de cada música, usando o nome da música como chave.
dic_div_tematica = {df.track_name[n]: {} for n in range(len(df))}

for i in range(num_fatias):
    fatia = df.iloc[i * tamanho_fatia:(i + 1) * tamanho_fatia]
    modelo_index = i  # Assumindo que você tem um modelo para cada fatia
    
    for index, row in fatia.iterrows():
        letra = row['letras_limpas'] 
        
        # Processa a letra, convertendo todas as palavras para minúsculas e removendo palavras de parada.
        letra_processada = [palavra.lower() for palavra in letra.split() if palavra.lower() not in stop_words]
        
        vetor_letra = media_vetores(letra_processada, modelos[modelo_index])
        
        # Obtem o resultado da similaridade temática entre a letra e os temas definidos.
        res = resultado(themes, modelos[modelo_index], vetor_letra)
        
        # Usar o nome correto da música
        dic_div_tematica[row['track_name']] = res 
        

### Passo 6: Analisar o desvio padrão dos temas

Como são muitos temas, para classificar a diversidade temática tiramos o desvio padrão de todos os temas. Abaixo, será apresentado um exemplo de como o dicionáro de temas se parece

Para fazer o desvio padrão, usamos a função `np.std` do numpy em um list comprehension e criamos uma coluna no dataset

In [ ]:
lista_desvios = [np.std(list(value.values())) for value in dic_div_tematica.values()]
df["diversidade tematica"] = lista_desvios

In [1010]:
df.head(5)

,artist,track_name,lyrics,genre,letras_limpas,Silabas,Frequencia,Pontuacao Gramatical,diversidade lexical,diversidade tematica
0,abba,0 - Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \nAnd...",pop,"look at her face, it's a wonderful face and ...",0.0,0.457516,0.71,2.848807,0.654861
1,abba,"1 - Andante, Andante","Take it easy with me, please \nTouch me gentl...",pop,"take it easy with me, please touch me gently...",0.0,0.307692,1.32,3.070545,0.576479
2,abba,2 - As Good As New,I'll never know why I had to go \nWhy I had t...,pop,i'll never know why i had to go why i had to...,0.0,0.362179,1.74,2.994859,0.615595
3,abba,3 - Bang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...,0.0,0.440000,1.03,3.135562,0.592457
4,abba,4 - Bang-A-Boomerang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...,0.0,0.454545,1.06,3.193389,0.602108


### 3.6 - Tamanho das letras

Este código cria um dicionário que associa o nome de cada música `track_name` ao tamanho de sua letra, medido pelo número de palavras. Ele percorre o dataFrame, conta as palavras nas letras limpas `letras_limpas` e armazena essa contagem no dicionário. Em seguida, esse dicionário é usado para criar uma nova coluna chamada Tamanho, mapeando o tamanho de cada letra com base no nome da faixa. Isso é útil para analisar a extensão das letras das músicas de forma eficiente.

In [ ]:
dic_tamanho = {row['track_name']: len(row['letras_limpas'].split()) for index, row in df.iterrows()}
df['Tamanho'] = df['track_name'].map(dic_tamanho)

In [1012]:
df.head(5)

,artist,track_name,lyrics,genre,letras_limpas,Silabas,Frequencia,Pontuacao Gramatical,diversidade lexical,diversidade tematica,Tamanho
0,abba,0 - Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \nAnd...",pop,"look at her face, it's a wonderful face and ...",0.0,0.457516,0.71,2.848807,0.654861,153
1,abba,"1 - Andante, Andante","Take it easy with me, please \nTouch me gentl...",pop,"take it easy with me, please touch me gently...",0.0,0.307692,1.32,3.070545,0.576479,260
2,abba,2 - As Good As New,I'll never know why I had to go \nWhy I had t...,pop,i'll never know why i had to go why i had to...,0.0,0.362179,1.74,2.994859,0.615595,312
3,abba,3 - Bang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...,0.0,0.440000,1.03,3.135562,0.592457,200
4,abba,4 - Bang-A-Boomerang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...,0.0,0.454545,1.06,3.193389,0.602108,198


### 3.7 - Repetição de versos:

Calcular a repetição de versos é importante para entender a estrutura das músicas e identificar padrões de repetição, que podem refletir estilos musicais, a simplicidade ou complexidade das letras, e até o impacto emocional das músicas. Músicas com muitos versos repetidos podem ser mais fáceis de lembrar ou cantar, enquanto músicas com menos repetição tendem a ser mais complexas e diversificadas. Esse cálculo fornece informaçoes sobre a composição.

### Passo 1: pontuar os versos

Esta função, pontuacao_verso, recebe um dicionário onde as chaves representam músicas e os valores são subdicionários que contêm contagens de versos. Para cada música, ela calcula o comprimento médio dos versos dividindo o total de versos pelo número de versos únicos. O resultado é armazenado em um novo dicionário, que mapeia cada música ao seu comprimento médio de versos. Isso ajuda a medir a complexidade ou repetição das letras das músicas com base no uso de versos únicos.

In [1013]:
def pontuacao_verso(dic_contagem_versos):
    dic = {}
    for key, value in dic_contagem_versos.items():
        total_versos = sum(value.values())  # Total de ocorrências de versos
        versos_unicos = len(value)  # Número de versos únicos
        
        # Calcula o comprimento médio dos versos
        comprimento_medio_versos = total_versos / versos_unicos if versos_unicos > 0 else 0
        dic[key] = comprimento_medio_versos
    
    return dic

Este código cria um dicionário, `dic_contagem_versos`, que armazena a contagem de cada verso em cada música presente no dataset

In [1014]:
dic_contagem_versos = {df.track_name[n]: Counter() for n in range(len(df))} 

# Itera sobre as linhas do DataFrame para contar os versos
for index, row in df.iterrows():
    versos = row['lyrics'].split('\n')  # Divide a letra em versos
    contagem_versos = Counter(verso.strip().lower() for verso in versos if verso.strip())
    
    #A função update soma as contagens de versos atuais com as novas contagens, garantindo que o total de ocorrências de cada verso seja corretamente acumulado ao longo do processo.
    dic_contagem_versos[row['track_name']].update(contagem_versos)
    
resultados = pontuacao_verso(dic_contagem_versos)

### Passo 2: criar coluna no dataset:

In [ ]:
df['Versos'] = df['track_name'].map(resultados)

In [1018]:
df.head(5)

,artist,track_name,lyrics,genre,letras_limpas,Silabas,Frequencia,Pontuacao Gramatical,diversidade lexical,diversidade tematica,Tamanho,Versos
0,abba,0 - Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \nAnd...",pop,"look at her face, it's a wonderful face and ...",0.0,0.457516,0.71,2.848807,0.654861,153,1.333333
1,abba,"1 - Andante, Andante","Take it easy with me, please \nTouch me gentl...",pop,"take it easy with me, please touch me gently...",0.0,0.307692,1.32,3.070545,0.576479,260,2.136364
2,abba,2 - As Good As New,I'll never know why I had to go \nWhy I had t...,pop,i'll never know why i had to go why i had to...,0.0,0.362179,1.74,2.994859,0.615595,312,1.434783
3,abba,3 - Bang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...,0.0,0.440000,1.03,3.135562,0.592457,200,1.307692
4,abba,4 - Bang-A-Boomerang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...,0.0,0.454545,1.06,3.193389,0.602108,198,1.307692


### 3.7 - Polaridade:

In [ ]:
polar = []
for palavra in df["letras_limpas"]:
    blob = TextBlob(palavra) 
    polar.append(blob.sentiment.polarity)

df["Polaridade"] = polar

In [1020]:
df.head(5)

,artist,track_name,lyrics,genre,letras_limpas,Silabas,Frequencia,Pontuacao Gramatical,diversidade lexical,diversidade tematica,Tamanho,Versos,Polaridade
0,abba,0 - Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \nAnd...",pop,"look at her face, it's a wonderful face and ...",0.0,0.457516,0.71,2.848807,0.654861,153,1.333333,0.447619
1,abba,"1 - Andante, Andante","Take it easy with me, please \nTouch me gentl...",pop,"take it easy with me, please touch me gently...",0.0,0.307692,1.32,3.070545,0.576479,260,2.136364,0.202222
2,abba,2 - As Good As New,I'll never know why I had to go \nWhy I had t...,pop,i'll never know why i had to go why i had to...,0.0,0.362179,1.74,2.994859,0.615595,312,1.434783,0.300881
3,abba,3 - Bang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...,0.0,0.440000,1.03,3.135562,0.592457,200,1.307692,0.355000
4,abba,4 - Bang-A-Boomerang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...,0.0,0.454545,1.06,3.193389,0.602108,198,1.307692,0.355000


## 4 - Classificação:

Como dito anteriormente, iremos calcular a complexidade das músicas através do modelo `RandomForestClassifier`, da biblioteca `sklearn`. Para treinar esse modelo, criamos um dataset consideradas complexas, chamado `dataset_complexo` e outro com músicas não complexas, chamado `dataset_nao_complexo`, os quais passaram pelos critérios acima. Para facilitar o entendimento, apresentaremos eles abaixo

### 4.1 - Apresentação dos dados:

In [1021]:
df_complexo = pd.read_csv("dataset_treino/dataset_Complexo.csv")
df_nao_complexo = pd.read_csv("dataset_treino/dataset_nao_complexo.csv")

### Passo 2 - Juntar os datasets:

In [1022]:
dataset_complexidade = pd.concat([df_complexo, df_nao_complexo], ignore_index=True)
dataset_complexidade.to_csv("Dataset_complexidade_2.csv", index=False)

### Passo 3 - Treinar o modelo:

In [1023]:
X = dataset_complexidade[['Silabas', 'Frequencia', 'Pontuacao Gramatical', 'diversidade lexical', 'diversidade tematica',  'Tamanho', 'Versos', 'Polaridade']]
y = dataset_complexidade['Complexidade']

Treinando o modelo:

In [1024]:
model = RandomForestClassifier()
model.fit(X, y)

RandomForestClassifier()

### Passo 4 - Classificar o dataset

Definir as colunas que serão classificadas:

In [1033]:
X_novo = df[['Silabas', 'Frequencia', 'Pontuacao Gramatical', 'diversidade lexical', 'diversidade tematica',  'Tamanho', 'Versos', 'Polaridade']]

In [1034]:
y_pred_novo = model.predict(X_novo)

### Passo 5 - Criar coluna complexidade:

In [ ]:
df['Complexidade'] = y_pred_novo

## 5 - Resultados:

In [1036]:
df.head(5)

,artist,track_name,lyrics,genre,letras_limpas,Silabas,Frequencia,Pontuacao Gramatical,diversidade lexical,diversidade tematica,Tamanho,Versos,Polaridade,Complexidade
0,abba,0 - Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \nAnd...",pop,"look at her face, it's a wonderful face and ...",0.0,0.457516,0.71,2.848807,0.654861,153,1.333333,0.447619,0
1,abba,"1 - Andante, Andante","Take it easy with me, please \nTouch me gentl...",pop,"take it easy with me, please touch me gently...",0.0,0.307692,1.32,3.070545,0.576479,260,2.136364,0.202222,0
2,abba,2 - As Good As New,I'll never know why I had to go \nWhy I had t...,pop,i'll never know why i had to go why i had to...,0.0,0.362179,1.74,2.994859,0.615595,312,1.434783,0.300881,0
3,abba,3 - Bang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...,0.0,0.440000,1.03,3.135562,0.592457,200,1.307692,0.355000,0
4,abba,4 - Bang-A-Boomerang,Making somebody happy is a question of give an...,pop,making somebody happy is a question of give an...,0.0,0.454545,1.06,3.193389,0.602108,198,1.307692,0.355000,0


In [1039]:
df.iloc[0:18].loc[df["Complexidade"] > 0]

,artist,track_name,lyrics,genre,letras_limpas,Silabas,Frequencia,Pontuacao Gramatical,diversidade lexical,diversidade tematica,Tamanho,Versos,Polaridade,Complexidade
6,abba,6 - Cassandra,Down in the street they're all singing and sho...,pop,down in the street they're all singing and sho...,0.0,0.418283,1.76,1.772483,1.090854,361,1.718750,-0.097061,1
11,abba,11 - Dancing Queen,"You can dance, you can jive, having the time o...",pop,"you can dance, you can jive, having the time o...",0.0,0.405941,0.84,3.399434,0.939233,202,1.500000,0.169307,1
12,abba,12 - Disillusion,"Changing, moving in a circle \nI can see your...",pop,"changing, moving in a circle i can see your ...",0.0,0.500000,0.75,3.229831,0.927320,164,1.312500,-0.022531,1
15,abba,15 - Dum Dum Diddle,"I can hear how you work, practising hard \nPl...",pop,"i can hear how you work, practising hard pla...",0.0,0.337719,1.22,2.095140,0.792441,228,1.714286,-0.213506,1
16,abba,16 - Eagle,"They came flying from far away, now I'm under ...",pop,"they came flying from far away, now i'm under ...",0.0,0.475676,0.88,3.361810,0.900845,185,1.157895,0.322068,1
